In [ ]:
import geopandas
import math
import pickle
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull,cKDTree
print("导包成功")


导包成功


In [ ]:
# 第一问求解
# 第一问
# 待求的各组起终点坐标值如下：
# （1）招商银行大厦总行营业部的坐标值：22.5369548317,114.0225501220
# （2）招商银行北京分行营业部的坐标值：39.9064024197,116.3624819407
# （3）招商银行兰州分行营业部的坐标值：36.0541745829,103.8366184893
# （4）招商银行上海分行营业部的坐标值：31.2427704453,121.5058705816
# （5）招商银行哈尔滨分行营业部的坐标值：45.7680708990, 126.6203889331
# 参赛选手需计算
# ：北京分行营业部到招商银行大厦距离i1
# ；兰州分行营业部到招商银行大厦距离i2
# ；上海分行营业部到招商银行大厦距离i3
# ；哈尔滨分行营业部到招商银行大厦距离i4


class Task1:
    @staticmethod
    def deg2rad(deg):
        return deg / 180 * math.pi

    @staticmethod
    def get_distance(position1, position2):
        # 定义求解距离公式
        a = 6378137  # 长轴
        f = 1 / 298.257222101  # 偏率
        b = (1 - f) * a  # 短轴
        e = math.sqrt(a * a - b * b) / a
        e_ = math.sqrt(a * a - b * b) / b
        
        B1, L1 = position1
        B2, L2 = position2
        u1 = math.atan(math.sqrt(1 - e * e) * math.tan(B1))
        u2 = math.atan(math.sqrt(1 - e * e) * math.tan(B2))
        delta_L = L2 - L1
        pre_lambda = delta_L
        while True:
            t1 = math.sin(pre_lambda) * math.cos(u2)
            t2 = math.cos(u1) * math.sin(u2) - math.sin(u1) * math.cos(u2) * math.cos(pre_lambda)
            sin_sigma = math.sqrt(t1 * t1 + t2 * t2)
    
            t1 = math.sin(u1) * math.sin(u2)
            t2 = math.cos(u1) * math.cos(u2) * math.cos(pre_lambda)
            cos_sigma = t1 + t2
    
            sin_alpha = math.cos(u1) * math.cos(u2) * math.sin(pre_lambda) / sin_sigma
            cos_alpha = math.sqrt(1 - sin_alpha * sin_alpha)
    
            cos_2_sigma_m = cos_sigma - (2 * math.sin(u1) * math.sin(u2) / (cos_alpha * cos_alpha))
    
            C = f / 16 * cos_alpha * cos_alpha * (4 + f * (4 - 3 * cos_alpha * cos_alpha))
    
            t1 = (1 - C) * f * sin_alpha
            t2 = (cos_2_sigma_m + C * cos_sigma * (2 * cos_2_sigma_m * cos_2_sigma_m - 1))
            t3 = math.asin(sin_sigma) + C * sin_sigma * t2
            now_lambda = delta_L + t1 * t3
            if abs(now_lambda - pre_lambda) <= 0.0000001:
                break
            else:
                pre_lambda = now_lambda
    
        k_2 = (e_ * cos_alpha) ** 2
        A = 1 + k_2 / 16384 * (4096 + k_2 * (k_2 * (320 - 175 * k_2) - 768))
        B = k_2 / 1024 * (256 + k_2 * (k_2 * (74 - 47 * k_2) - 128))
        t1 = B * sin_sigma
        t2 = cos_sigma * (2 * cos_2_sigma_m * cos_2_sigma_m - 1)
        t3 = cos_2_sigma_m * (4 * sin_sigma * sin_sigma - 3) * (4 * cos_2_sigma_m * cos_2_sigma_m - 3)
        delta_sigma = t1 * (cos_2_sigma_m + B / 4 * (t2 - B / 6 * t3))
        return b * A * (math.asin(sin_sigma) - delta_sigma)
    
    @staticmethod
    def get_task_answer():
        positions = [
            (22.5369548317, 114.0225501220),
            (39.9064024197, 116.3624819407),
            (36.0541745829, 103.8366184893),
            (31.2427704453, 121.5058705816),
            (45.7680708990, 126.6203889331),
        ]
        positions = list(map(lambda p: [Task1.deg2rad(p[0]), Task1.deg2rad(p[1])], 
                             positions))
        i1 = Task1.get_distance(positions[0], positions[1])
        i2 = Task1.get_distance(positions[0], positions[2])
        i3 = Task1.get_distance(positions[0], positions[3])
        i4 = Task1.get_distance(positions[0], positions[4])
        data = [i1, i2, i3, i4]
        return data


%time data1 = Task1.get_task_answer()
print(data1)


[1938531.4338824896, 1792997.2260526707, 1217053.529924019, 2818627.1666629612]


In [ ]:
class Task2:
    @staticmethod
    def get_nearest_osmid(G, position):
        osmid2position = G.osmid2position
        latitude, lontitude = position
        nearest_distance = np.inf
        nearest_osmid = 0
        for osmid, coords in osmid2position.items():
            distance = math.sqrt((coords[0] - latitude) ** 2 + (coords[1] - lontitude) ** 2)
            if distance < nearest_distance:
                nearest_distance = distance
                nearest_osmid = osmid
        return nearest_osmid
    
    @staticmethod
    def build_graph(n_path, e_path, weight="time"):
        node_shp_df = geopandas.GeoDataFrame.from_file(n_path)  # 读取nodes.shp数据
        edge_shp_df = geopandas.GeoDataFrame.from_file(e_path)  # 读取edges.shp数据
        G = nx.MultiDiGraph()
        osmid2position = {}
        osmids = node_shp_df['osmid']
        geometrys = node_shp_df['geometry']
        for i in range(len(node_shp_df)):
            osmid2position[osmids[i]] = tuple(reversed(geometrys[i].coords[0]))
            G.add_node(osmids[i])
    
        from_osmids = edge_shp_df['from'].values
        to_osmids = edge_shp_df['to'].values
        oneways = edge_shp_df['oneway'].values == 'True'
        
        if weight == "time":
            lengths = np.array(list(map(float, edge_shp_df['length'].values))) / np.array(
            list(map(float, edge_shp_df['base_speed'].values)))
        else:
            lengths = list(map(float, edge_shp_df['length'].values))
        
        for i in range(len(edge_shp_df)):
            from_osmid = from_osmids[i]
            to_osmid = to_osmids[i]
            oneway = oneways[i]
            length = lengths[i]
            if oneway:
                G.add_edge(from_osmid, to_osmid, length=length)
            else:
                G.add_edge(from_osmid, to_osmid, length=length)
                G.add_edge(to_osmid, from_osmid, length=length)
        G.osmid2position = osmid2position
        return G
    
    @staticmethod
    def get_task_answer():
            # 构建数据
            n_path = './data/FT_Camp_4/sandiego/nodes/nodes.shp'
            e_path = './data/FT_Camp_4/sandiego/edges/edges.shp'
            G = Task2.build_graph(n_path, e_path, weight="time")
        
            # 问题求解
            from2tos = [
                [(32.7496294, -117.1092834), (32.755474, -117.096519)],
                [(32.740292, -117.103363), (32.789948, -117.240753)],
                [(32.795853, -117.174637), (32.623028, -117.070381)],
                [(32.740292, -117.103363), (32.73925, -117.133629)],
                [(32.733253, -117.054108), (32.733768, -116.965523)],
                [(32.760967, -117.02327), (32.731846, -117.032837)],
                [(32.628593, -117.016388), (32.6183834, -117.1340124)],
                [(32.7507351, -117.1092739), (32.612331, -117.06707)],
                [(32.766434, -117.081886), (32.740261, -117.096588)],
                [(32.681469, -116.992996), (32.790749, -116.9315478)],
            ]
        
            data = []
            for from2to in from2tos:
                from_position = from2to[0]
                to_position = from2to[1]
                from_osmid = Task2.get_nearest_osmid(G, from_position)
                to_osmid = Task2.get_nearest_osmid(G, to_position)
                path_osmid = nx.shortest_path(G, from_osmid, to_osmid, weight="length")
                path = [list(G.osmid2position[osmid]) for osmid in path_osmid]
                data.append(path)
            return data


%time data2 = Task2.get_task_answer()
print(data2[0])


In [ ]:
# 方案1 考虑强约束中断条件
# 方案2 不考虑单行道条件
# 方案3 不考虑强中断条件


class ConcaveHull(object):
    def __init__(self, points, k=3):
        self.points = points
        self.temp_list = np.copy(self.points)
        self.k = k
        self.vertices = list()

        self.start_point = self.find_start_point()
        self.add_point_to_vertices(self.start_point)
        self.compute_vertices()
        return

    def compute_vertices(self):
        while True:
            dists, idxs = self.find_k_nearest_neighbour(self.points[self.vertices[-1]])
            idxs = np.delete(idxs, np.where(dists == np.inf))
            next_points, indexes = self.find_next_point(idxs, self.get_base_points())
            next_point = self.find_intersections(next_points)

            if (next_point[0] == self.start_point[0] and
                    next_point[1] == self.start_point[1]):
                self.add_point_to_vertices(self.start_point)
                break

            self.add_point_to_vertices(next_point)

            if len(self.vertices) == 4:
                self.temp_list = np.append(self.temp_list, [self.start_point], 0)
        return

    def find_start_point(self):
        min_value = np.min(self.points[:, 1])
        return self.points[np.where(self.points[:, 1] == min_value)][0]

    def find_k_nearest_neighbour(self, point):
        self.temp_list = self.exclude_point_from_list(point, self.temp_list)
        tree = cKDTree(self.temp_list.copy())
        return tree.query(point, k=self.k)

    def find_next_point(self, indexes, base_points):
        base_vector = self.get_vector_from_two_point(base_points[0], base_points[1])
        angles = np.array([])
        candidates = self.temp_list[indexes]
        for point in candidates:
            vector = self.get_vector_from_two_point(base_points[0], point)
            angle = np.arctan2(np.cross(base_vector, vector), np.dot(base_vector, vector))
            angle = angle if angle < 0 else (-np.pi - (np.pi - angle))
            angles = np.append(angles, angle)
        idxs = np.argsort(angles)
        return candidates[idxs], indexes[idxs]

    def find_index_of_point(self, point):
        return np.where(np.logical_and(self.points[:, 0] == point[0],
                                       self.points[:, 1] == point[1]))[0][0]

    def add_point_to_vertices(self, point):
        idx = self.find_index_of_point(point)
        self.vertices.append(idx)
        return

    def find_intersections(self, point_list):
        if len(self.vertices) >= 3:
            for point in point_list:
                curr_segment = [self.points[self.vertices[-1]], point]
                intersects = False
                for i in range(len(self.vertices) - 1):
                    segment = [self.points[self.vertices[i]], self.points[self.vertices[i + 1]]]
                    if (self.check_segment_intersections(curr_segment, segment) and
                            not self.check_same_points_in_lines(curr_segment, segment)):
                        intersects = True
                        break
                if not intersects:
                    return point
        return point_list[0]

    def get_base_points(self):
        if len(self.vertices) == 1:
            base_points = [self.start_point, np.array([self.start_point[0] - 1, self.start_point[1]])]
        else:
            base_points = [self.points[self.vertices[-1]], self.points[self.vertices[-2]]]
        return base_points

    @staticmethod
    def get_vector_from_two_point(start_point, end_point):
        return np.array([(end_point[0] - start_point[0]), (end_point[1] - start_point[1])])

    @staticmethod
    def exclude_point_from_list(point, point_list):
        return np.delete(point_list, np.where(np.logical_and(point_list[:, 1] == point[1],
                                                             point_list[:, 0] == point[0])),
                         axis=0)

    @staticmethod
    def check_segment_intersections(line1, line2):
        r = (line1[1][0] - line1[0][0], line1[1][1] - line1[0][1])
        s = (line2[1][0] - line2[0][0], line2[1][1] - line2[0][1])
        cross_prod = np.cross(r, s)
        if cross_prod == 0:
            return False
        diff = (line2[0][0] - line1[0][0], line2[0][1] - line1[0][1])
        t = np.cross(diff, s) / cross_prod
        u = np.cross(diff, r) / cross_prod
        if (0 <= t <= 1) and (0 <= u <= 1):
            return True
        return False

    @staticmethod
    def check_same_points_in_lines(line1, line2):
        return ((line1[0][0] == line2[0][0] and line1[0][1] == line2[0][1]) or
                (line1[0][0] == line2[1][0] and line1[0][1] == line2[1][1]) or
                (line1[1][0] == line2[0][0] and line1[1][1] == line2[0][1]) or
                (line1[1][0] == line2[1][0] and line1[1][1] == line2[1][1]))


class Task3(Task2):
    @staticmethod
    def get_service_position_set1(center, G, threshold):
        service_position_set = []
        center_osmid = Task3.get_nearest_osmid(G, center)
        for node_osmid, length in nx.single_source_dijkstra_path_length(G, center_osmid, weight="length",
                                                                        cutoff=threshold).items():
            service_position_set.append(G.osmid2position[node_osmid])
        return service_position_set
    
    @staticmethod
    def get_service_position_set2(center, G, threshold):
        service_position_set = []
        service_position_distance = {}
        center_osmid = Task3.get_nearest_osmid(G, center)
        for node_osmid, length in nx.single_source_dijkstra_path_length(G, center_osmid, weight="length",
                                                                        cutoff=threshold).items():
            service_position_distance[node_osmid] = length
            service_position_set.append(G.osmid2position[node_osmid])
    
        service_position_distance2 = {}
        for node_osmid, length in nx.single_source_dijkstra_path_length(G, center_osmid, weight="length",
                                                                        cutoff=2 * threshold).items():
            service_position_distance2[node_osmid] = length
    
        for outer_node_osmid in service_position_distance2.keys():
            if outer_node_osmid not in service_position_distance:
                for inside_node_osmid in G.predecessors(outer_node_osmid):
                    if inside_node_osmid in service_position_distance and \
                       2 * threshold > service_position_distance[inside_node_osmid] + service_position_distance2[outer_node_osmid]:
                        service_position_set.append(G.osmid2position[outer_node_osmid])
                        break
        return service_position_set
    
    @staticmethod
    def get_service_position_set3(center, G, threshold):
        service_position_set = []
        service_position_distance = {}
        center_osmid = Task3.get_nearest_osmid(G, center)
        for node_osmid, length in nx.single_source_dijkstra_path_length(G, center_osmid, weight="length",
                                                                        cutoff=threshold).items():
            service_position_distance[node_osmid] = length
            service_position_set.append(G.osmid2position[node_osmid])
    
        service_position_distance2 = {}
        for node_osmid, length in nx.single_source_dijkstra_path_length(G, center_osmid, weight="length",
                                                                        cutoff=2 * threshold).items():
            service_position_distance2[node_osmid] = length
    
        for outer_node_osmid in service_position_distance2.keys():
            if outer_node_osmid not in service_position_distance:
                for inside_node_osmid in G.predecessors(outer_node_osmid):
                    if inside_node_osmid in service_position_distance:
                        outer_position = G.osmid2position[outer_node_osmid]
                        inside_position = G.osmid2position[inside_node_osmid]
                        a = (threshold - service_position_distance[inside_node_osmid])
                        b = (service_position_distance2[outer_node_osmid] -service_position_distance[inside_node_osmid])
                        k = a / b
                        lat = k * (outer_position[0] - inside_position[0]) + inside_position[0]
                        lon = k * (outer_position[1] - inside_position[1]) + inside_position[1]
                        service_position_set.append([lat, lon])
        return service_position_set
    
    @staticmethod
    def get_convex_hull(points):
        convex_hull = ConvexHull(points)
        convex_point_set = []
        vertices = list(convex_hull.vertices)
        vertices.append(vertices[0])
        for idx in vertices:
            convex_point_set.append([points[idx][0], points[idx][1]])
        return convex_point_set
    
    @staticmethod
    def get_concave_hull(points, k=30):
        service_position_set = np.array(points)
        concave_hull = ConcaveHull(service_position_set, k=k)
        concave_point_set = [[x_y[0], x_y[1]] for x_y in service_position_set[concave_hull.vertices, :]]
        return concave_point_set
    
    @staticmethod
    def get_task_answer():
        # 构建数据
        n_path = './data/FT_Camp_4/sandiego/nodes/nodes.shp'
        e_path = './data/FT_Camp_4/sandiego/edges/edges.shp'
        G = Task2.build_graph(n_path, e_path, weight="length")
    
        # 问题求解
        centers = [
            (32.766434, -117.081886),
            (32.7590688, -117.1618495),
            (32.733253, -117.054108),
            (32.623028, -117.070381),
        ]
        data = []
        service_position_sets = []
        for threshold in [1000.0, 3000.0]:
            for center in centers:
                # service_position_set = Task3.get_service_position_set1(center, G, threshold)  # 不考虑扩充点
                # service_position_set = Task3.get_service_position_set2(center, G, threshold)   # 考虑扩充点
                service_position_set = Task3.get_service_position_set3(center, G, threshold) # 考虑路中的点
                service_position_sets.append(service_position_set)
                
                if threshold < 2000:  # 凸包的求解
                    data.append(Task3.get_convex_hull(service_position_set))
                else:  # 凹包的求解
                    data.append(Task3.get_concave_hull(service_position_set, 30))
        return data, service_position_sets
        

%time data3, service_position_sets = Task3.get_task_answer()
print(data3[0])


In [ ]:
class Task4:
    @staticmethod
    def get_distance(position1, position2):
        return math.sqrt((position1[0] - position2[0]) ** 2 + (position1[1] - position2[1]) ** 2) * 100

    @staticmethod
    def get_resident_positions(resident_path):
        resident = pd.read_csv(resident_path)
        home_positions = resident['home'].values
        home_positions = [tuple(map(float, home_position[1:-1].split(','))) for home_position in home_positions]
        company_positions = resident['company'].values
        company_positions = [tuple(map(float, company_position[1:-1].split(','))) for company_position in company_positions]
        return home_positions, company_positions

    @staticmethod
    def get_area_human_number(positions):
        tree = cKDTree(np.array(positions))
        human_count = {}
        for position in positions:
            human_count[position] = len(tree.query_ball_point(position, r=0.02, n_jobs=6))
        return human_count

    @staticmethod
    def get_increase_ratio(v1, v2):
        r = 0
        for i in range(5):
            r += ((v2[i][-1] - v1[i][-1]) / v1[i][-1])
        return r

    @staticmethod
    def iterative_greedy(positions, human_count):
        answer = []
        for i in range(1000):  # 改变初始体条件的贪心搜索
            t_answer = [[positions[i][0], positions[i][1], human_count[positions[i]]]]
            for j in range(i+1, len(positions)):
                for p in t_answer:
                    if Task4.get_distance(p, positions[j]) < 4.0:
                        break
                else:
                    t_answer.append([positions[j][0], positions[j][1], human_count[positions[j]]])
                if len(t_answer) == 5:
                    break
                    # t_answer[-1][-1] > answer[-1][-1] best
            if len(answer) == 0 or Task4.get_increase_ratio(answer, t_answer) > 0:  
                answer = t_answer
        return answer

    @staticmethod
    def get_task_answer():
        # 准备数据
        home_positions, company_positions = Task4.get_resident_positions('./data/FT_Camp_4/resident.csv')
        # 不同地区人员统计
        human_count_in_home = Task4.get_area_human_number(home_positions)
        human_count_in_company = Task4.get_area_human_number(company_positions)
        
        #  利用贪心思想求解结果
        data = []
        home_positions.sort(key=lambda p: -human_count_in_home[p])
        company_positions.sort(key=lambda p: -human_count_in_company[p])
    
        home_answer = Task4.iterative_greedy(home_positions, human_count_in_home)
        company_answer = Task4.iterative_greedy(company_positions, human_count_in_company) 
    
        data.extend(home_answer)
        data.extend(company_answer)
        return data
    
    
%time data4 = Task4.get_task_answer()
print(data4)


In [ ]:
# 将结果保存至submit.txt文件，之后用于提交
def write_to_txt(data1, data2, data3, data4, path='./submit.txt'):
    def format_data_list(data):
        return '\n'.join([str(item) for item in data])

    def format_data_str(data):
        coor_str = ''
        for item in data:
            coor_str += ','.join([str(coor) for coor in item]) + '\n'
        return coor_str

    def format_data():
        split_str = '\n=====\n'
        # 任务一
        output = ','.join([str(u) for u in data1])
        # 任务二
        output += split_str
        output += format_data_list(data2)
        # 任务三
        output += split_str
        output += format_data_list(data3)
        # 任务四
        output += split_str
        output += format_data_str(data4)
        return output

    if len(data1) != 4 or len(data2) != 10 or len(data3) != 8 or len(data4) != 10:
        return False
    try:
        with open(path, 'w+') as f:
            split_str = '\n=====\n'
            f.write(format_data())
        print('生成提交结果文件成功!')
    except Exception as e:
        print('读写文件失败', e)
        
        
write_to_txt(data1, data2, data3, data4, "./submit.txt")


In [ ]:
import xlab
xlab.ftcamp.submit('./submit.txt')

In [ ]:
xlab.ftcamp.get_submit_hist()

In [ ]:
def plot_data3_result(ids):
    import json
    if min(ids) < 0 or max(ids) > len(data3):
        raise Exception("请输入一个0到{0}".format(len(data3) - 1))
    centers = [
        (32.766434, -117.081886),
        (32.7590688, -117.1618495),
        (32.733253, -117.054108),
        (32.623028, -117.070381),
        (32.766434, -117.081886),
        (32.7590688, -117.1618495),
        (32.733253, -117.054108),
        (32.623028, -117.070381),
    ]

    e_path = './data/FT_Camp_4/sandiego/edges/edges.shp'
    edge_shp_df = geopandas.GeoDataFrame.from_file(e_path)  # 读取edges.shp数据
    
    area = [[] for _ in range(8)]
    for i in range(1, 5):
        with open("./arcgis_km/{0}.json".format(i)) as f:
            d = json.load(f)
        for j in range(2):
            locations = d["saPolygons"]["features"][j]["geometry"]["rings"][0]
            area[4*(0 if j == 1 else 1)+i-1] = locations

    fig, ax = plt.subplots(figsize=(30, 30))
    for i in ids:
        service_position_set = service_position_sets[i]
        center = centers[i]
        x = [x_y[0] for x_y in service_position_set]
        y = [x_y[1] for x_y in service_position_set]
        ax.scatter(y, x, color='green', s=50)
        x = [x_y[0] for x_y in data3[i]]
        y = [x_y[1] for x_y in data3[i]]
        ax.plot(y, x, linewidth=5, color='red')
        x = [x_y[1] for x_y in area[i]]
        y = [x_y[0] for x_y in area[i]]
        ax.plot(y, x, linewidth=5, color="black")
        ax.scatter(center[1], center[0], color='red', s=300)
#     edge_shp_df.plot(ax=ax, color='black')
    plt.show()


plot_data3_result([2, 3, 6, 7])
